In [1]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferMemory()

memory.save_context({"input": "Hi!"}, {"output": "Hello!"})

memory.load_memory_variables({})

{'history': 'Human: Hi!\nAI: Hello!'}

In [4]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hi!"}, {"output": "Hello!"})
memory.load_memory_variables({})
memory.save_context({"input": "Hi!"}, {"output": "Hello!"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='Hello!'),
  HumanMessage(content='Hi!'),
  AIMessage(content='Hello!')]}

In [ ]:

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

ConversationBufferWindowMemory

In [9]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

add_message(1, 1)
add_message(2 ,2)
add_message(3, 3)
add_message(4, 4)
memory.load_memory_variables({})


{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [8]:

add_message(5, 5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

ConversationSummaryMemory

In [11]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


add_message("Hi, I'm Nicolas. I live in South Korea", "Wow! that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")
get_history()

{'history': "The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds with enthusiasm, saying that it thinks it's cool and expresses a desire to visit because it thinks South Korea is pretty."}

ConversationSummaryBufferMemory

In [20]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()


{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea.'),
  AIMessage(content='Wow that is so cool!')]}

In [21]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()


{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds with enthusiasm, saying that it is cool. The human adds that South Korea is very pretty.'),
  AIMessage(content='I wish I could go!!!')]}

In [22]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()


{'history': [SystemMessage(content='The human introduces themselves as Nicolas and mentions that they live in South Korea, saying that it is very pretty. The AI responds with enthusiasm, saying that it is cool and expressing a desire to go. The human then asks how far Korea is from Argentina.'),
  AIMessage(content="I don't know! Super far!")]}

In [23]:

add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
get_history()

{'history': [SystemMessage(content="The human introduces themselves as Nicolas and mentions that they live in South Korea, describing it as very pretty. The AI responds with enthusiasm, expressing a desire to go and saying that it is cool. The human then asks how far Korea is from Argentina. The AI admits that it doesn't know, but thinks it is super far. The human then asks how far Brazil is from Argentina."),
  AIMessage(content="I don't know! Super far!")]}

ConversationKGMemory

In [24]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas?"})


{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}

In [27]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"input": "what does nicolas like?"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [28]:
memory.load_memory_variables({"input": "what is Kimchi?"})

{'history': []}

In [29]:
memory.load_memory_variables({"input": "what is South Korea?"})

{'history': []}

In [30]:
memory.load_memory_variables({"input": "what is so cool?"})

{'history': []}

In [31]:
memory.load_memory_variables({"input": "Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [32]:
add_message("Hi I'm Daniel, I live in Canada", "Wow that is also so cool!")

In [33]:
memory.load_memory_variables({"input": "Daniel"})

{'history': [SystemMessage(content='On Daniel: Daniel is Human. Daniel lives in Canada.')]}

In [34]:
memory.load_memory_variables({"input": "Nicolas and Daniel"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.'),
  SystemMessage(content='On Daniel: Daniel is Human. Daniel lives in Canada.')]}

In [35]:
memory.load_memory_variables({"input": "Relationship of Nicolas and Daniel"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.'),
  SystemMessage(content='On Daniel: Daniel is Human. Daniel lives in Canada.')]}

In [37]:
# Just extract entities and send responses
memory.load_memory_variables({"input": "Does Nicolas like Daniel?"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.'),
  SystemMessage(content='On Daniel: Daniel is Human. Daniel lives in Canada.')]}

ConversationEntityMemory

In [52]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory
llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct"
)

memory = ConversationEntityMemory(
    llm=llm,
    k=2
)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)


{'history': '', 'entities': {'Deven': '', 'Sam': ''}}

In [53]:
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

memory.load_memory_variables({"input": 'who is Sam?'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [54]:
memory.save_context(
    {"input": "What kind of project do they work on?"},
    {"output": "They are working on online ticket issuing system."}
)

memory.load_memory_variables({"input": 'project, Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?\nHuman: What kind of project do they work on?\nAI: They are working on online ticket issuing system.',
 'entities': {'Sam': 'Sam is working on an online ticket issuing system with Deven for a hackathon project.'}}

In [55]:
memory.load_memory_variables({"input": 'Deven, Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?\nHuman: What kind of project do they work on?\nAI: They are working on online ticket issuing system.',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam.',
  'Sam': 'Sam is working on an online ticket issuing system with Deven for a hackathon project.'}}

In [56]:
memory.save_context(
    {"input": "When is due of the project?"},
    {"output": "The end of this week."}
)

memory.load_memory_variables({"input": 'Deven, Sam'})

{'history': 'Human: What kind of project do they work on?\nAI: They are working on online ticket issuing system.\nHuman: When is due of the project?\nAI: The end of this week.',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam and the project is due at the end of this week.',
  'Sam': "Sam's project is due at the end of this week."}}

In [59]:
memory.save_context(
    {"input": "When is the name of hackathon?"},
    {"output": "The project name is 'Be smart! Be lazy!'"}
)

memory.load_memory_variables({"input": 'Deven, Sam, hackathon, project'})

{'history': "Human: When is the name of hackathon?\nAI: 'Be smart! Be lazy!'\nHuman: When is the name of hackathon?\nAI: The project name is 'Be smart! Be lazy!'",
 'entities': {'Deven': "Deven is working on a hackathon project with Sam and the project is due at the end of this week. The name of the hackathon is 'Be smart! Be lazy!'.",
  'Sam': "Sam is participating in a hackathon called 'Be smart! Be lazy!'.",
  'hackathon': "The project name is 'Be smart! Be lazy!'",
  'project': "The project name is 'Be smart! Be lazy!'."}}

ConversationTokenBufferMemory

In [60]:
from langchain.memory import ConversationTokenBufferMemory

llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct"
)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}